In [1]:
import gspread
import pandas as pd
import numpy as np

In [2]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('temporary')

# we also need to specify the page name before getting the data.
rxns_sheet = cho_recon.worksheet('Rxns')

In [3]:
for sheets in cho_recon:
    print(sheets)

<Worksheet 'Info' id:0>
<Worksheet 'Rxns' id:1966089892>
<Worksheet 'Attributes' id:745769606>
<Worksheet 'copyAttributes' id:368082576>
<Worksheet 'Added Rxns' id:1377582373>
<Worksheet 'Genes' id:239167986>


In [4]:
# We can extract the data using the get_all_records method and create a pd DataFrame
df = pd.DataFrame(rxns_sheet.get_all_records())
df = df.set_index('Index')
df

,Curated,Reaction,Reaction Name,Reaction Formula,Subsystem,GPR_hef,GPR_fou,GPR_yeo,GPR_Recon3D,GPR_final,GPR_Final_Thanasis,Conf. Score,Curation Notes,References
Index,,,,,,,,,,,,,,
0,PD,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal",10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
1,PD,10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial",10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,,1,No information available in the literature abo...,
2,PD,10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal",10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
3,PD,10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial",10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,,1,No information available in the literature abo...,
4,PD,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal",10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8195,,r2534,Major Facilitator(MFS) TCDB:2.A.1.44.1,thr_L_e <=> thr_L_c,"TRANSPORT, EXTRACELLULAR",,,100757617,100757617,,100757617,,,
8196,,r2535,Major Facilitator(MFS) TCDB:2.A.1.44.1,hom_L_e <=> hom_L_c,Transport,,,100757617,100757617,,100757617,,,
8197,,r2537,Utilized transport,lnlncgcoa_c <=> lnlncgcoa_r,Transport,,,,,,,,,


In [5]:
import re

gene_list = []
for index, row in df.iterrows():
    if row['GPR_Final_Thanasis'] != '':
        gpr = str(row['GPR_Final_Thanasis'])
        num = re.findall(r'\d+', gpr)
        for n in num:
            gene_list.append(n)
        
gene_list = list(set(gene_list))

In [6]:
# Fetch information from the NIH database

In [7]:
from Bio import Entrez

def get_gene_info(gene_id):
    '''
    This function retrieves information from the NIH Gene database
    
    input: Gene Entrez ID
    output: Gene Symbol, Gene Name, Gene Description, Gene Ensembl ID, NCBI Transcript ID, NCBI Protein ID
    '''
    Entrez.email = 'account1@theta-ocean-377718.iam.gserviceaccount.com'
    handle = Entrez.efetch(db='gene', id=gene_id, retmode='xml')
    record = Entrez.read(handle)[0]

    gene_name = record['Entrezgene_gene']['Gene-ref']['Gene-ref_desc']
    gene_symbol = record['Entrezgene_gene']['Gene-ref']['Gene-ref_locus']

    # check for different possible formats of the data
    if 'Entrezgene_comments' in record and 'Gene-commentary_comment' in record['Entrezgene_comments'][0]:
        gene_description = record['Entrezgene_comments'][0]['Gene-commentary_comment'][0]['String']
    elif 'Entrezgene_summary' in record:
        gene_description = record['Entrezgene_summary']
    else:
        gene_description = None

    if 'Entrezgene_track-info' in record:
        gene_ensembl_id = next((xref['Dbtag_tag']['Object-id']['Object-id_str'] for xref in record['Entrezgene_gene']['Gene-ref']['Gene-ref_db'] if xref['Dbtag_db'] == 'Ensembl'), None)
    else:
        gene_ensembl_id = None

    xrefs = record['Entrezgene_locus'][0]['Gene-commentary_products']
    
    for xref in xrefs:
        if xref.get('Gene-commentary_accession').startswith('NM_'):
            mRNA_ncbi_id = xref.get('Gene-commentary_accession')
            protein_ncbi_id = xref['Gene-commentary_products'][0].get('Gene-commentary_accession')
            break
        elif xref.get('Gene-commentary_accession').startswith('XM_'):
            mRNA_ncbi_id = xref.get('Gene-commentary_accession')
            protein_ncbi_id = xref['Gene-commentary_products'][0].get('Gene-commentary_accession')
            break
    

    handle.close()

    return gene_symbol, gene_name, gene_description, gene_ensembl_id, mRNA_ncbi_id, protein_ncbi_id


In [ ]:
'''
import time

# Open the Genes excel Sheet
cho_temporary= sa.open('temporary')
copyattributes_sheet = cho_temporary.worksheet('Genes')

id = 2
# Read genes for the .txt file
with open('GeneOutput.txt') as file:  
    for gene_id in file:
        try:
            gene_symbol, gene_name, gene_description, gene_ensembl_id, mRNA_ncbi_id, protein_ncbi_id = get_gene_info(gene_id)
            print(id)
            copyattributes_sheet.update_cell(id,1,id-1)
            time.sleep(5)
            copyattributes_sheet.update_cell(id,2,gene_id)
            time.sleep(5)
            copyattributes_sheet.update_cell(id,3,gene_symbol)
            time.sleep(5)
            copyattributes_sheet.update_cell(id,4,gene_name)
            time.sleep(5)
            copyattributes_sheet.update_cell(id,5,gene_description)
            time.sleep(5)
            copyattributes_sheet.update_cell(id,6,gene_ensembl_id)
            time.sleep(5)
            copyattributes_sheet.update_cell(id,7,mRNA_ncbi_id)
            time.sleep(5)
            copyattributes_sheet.update_cell(id,8,protein_ncbi_id)
            id += 1
        except:
            print('Google API quota exceeded')
            time.sleep(5)
            continue
'''

In [ ]:
import time

# Open the Genes excel Sheet
cho_temporary= sa.open('temporary')
genes_sheet = cho_temporary.worksheet('Genes')

#id = 2
# Read genes for the .txt file

for gene in gene_list:
    df = pd.DataFrame(genes_sheet.get_all_records())
    # the first try/except is to avoid overwritting data in case there already some info in the dataset
    try:
        gene_sheet_list = list(df['Gene Entrez ID'])
        id = max(df['Index']) + 2
    except:
        gene_sheet_list = []
        id = 2
    if gene not in gene_sheet_list:
        try:
            gene_symbol, gene_name, gene_description, gene_ensembl_id, mRNA_ncbi_id, protein_ncbi_id = get_gene_info(gene)
            print(id)
            genes_sheet.update_cell(id,1,id-1)
            time.sleep(5)
            genes_sheet.update_cell(id,2,gene)
            time.sleep(5)
            genes_sheet.update_cell(id,3,gene_symbol)
            time.sleep(5)
            genes_sheet.update_cell(id,4,gene_name)
            time.sleep(5)
            genes_sheet.update_cell(id,5,gene_description)
            time.sleep(5)
            genes_sheet.update_cell(id,6,gene_ensembl_id)
            time.sleep(5)
            genes_sheet.update_cell(id,7,mRNA_ncbi_id)
            time.sleep(5)
            genes_sheet.update_cell(id,8,protein_ncbi_id)
            #id += 1
        except:
            print('Google API quota exceeded')
            time.sleep(5)
            continue

Google API quota exceeded
2
2
Google API quota exceeded
3
4
5
6
Google API quota exceeded
7
8
9
10
11
12
13
Google API quota exceeded
14
15
16
17
18
19
20
21
Google API quota exceeded
22
23
24
Google API quota exceeded
25
26
27
28
29
30
31
32


In [ ]:
df = pd.DataFrame(genes_sheet.get_all_records())
df

In [ ]:
cho_temporary= sa.open('temporary')
genes_sheet = cho_temporary.worksheet('Genes')
df = pd.DataFrame(genes_sheet.get_all_records())
id = max(df['Index'])
id_list